In [1]:
# Import libraries 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone

/Users/visakh/Desktop/Gen_AI/medical_chat_bot/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Pinecone_API_KEY = 'pcsk_4SBxDE_LZKJYBsmo7dHqPrADDhgGkhLPphsNcmZu2ftxCUxsNp8Z8MQTT5TXh5L36PX3FE'

In [3]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents



In [4]:
extracted_data = load_pdf('/Users/visakh/Desktop/Gen_AI/medical_chat_bot/Data/')

In [5]:
# Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk


In [6]:
text_chunks = text_split(extracted_data)

In [7]:
len(text_chunks)

40090

In [8]:
# Create an embedding model 
def download_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
from sentence_transformers import SentenceTransformer

In [10]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [11]:
import chromadb

In [12]:
!pip show chromadb

Name: chromadb
Version: 0.6.3
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /Users/visakh/Desktop/Gen_AI/medical_chat_bot/.env/lib/python3.12/site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, rich, tenacity, tokenizers, tqdm, typer, typing_extensions, uvicorn
Required-by: 


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb

class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

    def __call__(self, input):
        return self.embeddings_model.embed_documents(input)
    
    def embed_documents(self, texts):
        return self.embeddings_model.embed_documents(texts)

    def embed_query(self, text):
        return self.embeddings_model.embed_query(text)




embeddings = ChromaEmbeddingWrapper("sentence-transformers/all-MiniLM-L6-v2")

# Use PersistentClient
client = chromadb.PersistentClient(path="./chroma_langchain_db")

vector_store = Chroma(
    client=client,
    collection_name="medical_chat",
    embedding_function=embeddings,
)


ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/guides/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/deployment/migration#migration-to-0.4.16---november-7,-2023 
